Import

In [1]:
import os, random
import numpy as np
import pandas as pd

from ase.io import read

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader

from tqdm.auto import tqdm

np.set_printoptions(threshold=np.inf)

In [2]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)

seed_everything(42) # Seed 고정 : 항상 같은 조건에서 실험을 수행하기 위하여 설정함.

In [3]:
!pwd

'pwd'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


Pre-Processing

In [4]:
# Train과 Test 데이터셋을 'extxyz' 포맷으로 불러옴.
train = read('./data/train.xyz', format='extxyz', index=':') # 전체 데이터 불러오기
test = read('./data/test.xyz', format='extxyz', index=':')
sample = pd.read_csv('./data/sample_submission.csv')

In [5]:
print(f"The number of data: {len(train)}")
train[0]

The number of data: 22510


Atoms(symbols='N24Si24', pbc=True, cell=[8.52238831, 8.52238831, 8.52238831], forces=..., calculator=SinglePointCalculator(...))

In [6]:
sample

,ID,energy,force
0,TEST_0000,0,[[0. 0. 0.]\n [0. 0. 0.]\n [0. 0. 0.]\n [0. 0....
1,TEST_0001,0,[[0. 0. 0.]\n [0. 0. 0.]\n [0. 0. 0.]\n [0. 0....
2,TEST_0002,0,[[0. 0. 0.]\n [0. 0. 0.]\n [0. 0. 0.]\n [0. 0....
3,TEST_0003,0,[[0. 0. 0.]\n [0. 0. 0.]\n [0. 0. 0.]\n [0. 0....
4,TEST_0004,0,[[0. 0. 0.]\n [0. 0. 0.]\n [0. 0. 0.]\n [0. 0....
...,...,...,...
4096,TEST_4096,0,[[0. 0. 0.]\n [0. 0. 0.]\n [0. 0. 0.]\n [0. 0....
4097,TEST_4097,0,[[0. 0. 0.]\n [0. 0. 0.]\n [0. 0. 0.]\n [0. 0....
4098,TEST_4098,0,[[0. 0. 0.]\n [0. 0. 0.]\n [0. 0. 0.]\n [0. 0....
4099,TEST_4099,0,[[0. 0. 0.]\n [0. 0. 0.]\n [0. 0. 0.]\n [0. 0....


In [15]:
sample.to_csv('sample.csv')

# 분자 에너지
- 분자의 에너지는 분자의 내부 상태와 상호작용에 따른 총 에너지를 의미합니다.
- 에너지의 단위는 일반적으로 전자볼트(electronvolt, eV) 등이 사용됩니다.
- 분자의 에너지는 원자들 간의 결합, 전하 상호작용, 분자 내부 진동 등 다양한 요인들에 의해 결정됩니다.
- 분자의 안정성과 화학 반응 경로 등을 이해하기 위해 에너지 정보가 필요합니다.

# 힘(Force)
- 분자 내부에서 원자들 사이에 작용하는 힘은 분자의 구조와 움직임을 결정짓는 중요한 요소입니다.
- 힘의 단위는 뉴턴(Newton) 등의 단위가 사용됩니다.
- 분자의 힘은 분자 내부의 에너지 표면을 나타내는데 사용됩니다. 에너지 표면은 분자의 구조에 따른 에너지 변화를 보여주는 것으로, 최적화나 동역학 시뮬레이션 등에서 활용됩니다.
- 또한, 분자 간 상호작용이나 반응 경로 등에서 힘의 정보가 필요합니다.

# 시도해 볼만 한 모델
힘 : GPR <br>
에너지 : GNN(Kaggle에 있음)

In [7]:
# train
# 각 분자의 원자개수, 위치, 힘, 에너지 정보 추출
sequence_train, symbols, positions_x, positions_y, positions_z, forces, energies = [], [], [], [], [], [], []

for i in range(len(train)):
    mole = train[i] # 각 분자

    atoms = len(mole) # 현재 분자의 원자 개수
    sequence_train.append(atoms) # 해당 분자의 원자 개수를 저장

    position = mole.get_positions() # 원자 위치 정보
    force = mole.get_forces() # label 1 : 해당 분자의 힘 정보를 저장

    energy = mole.get_total_energy() # label 2 : 해당 분자의 총 에너지 정보를 저장
    energies.append(energy)

    for j in range(len(mole)): # 각 원자에 대해
        atom = mole[j]  # atom 변수에 mole[j]번째 원자를 할당

        # 해당 원자의 x, y, z 좌표 값을 각각 추가.
        positions_x.append(position[j][0])
        positions_y.append(position[j][1])
        positions_z.append(position[j][2])
        # 해당 원자의 힘 정보를 forces 리스트에 추가
        forces.append(force[j])

# 각 분자의 원자 위치와 힘 정보를 정리한 형태로 저장됨.
train_df = pd.DataFrame({'position_x': positions_x, 'position_y':positions_y, 'position_z':positions_z, 'force':forces})
train_df.head()

,position_x,position_y,position_z,force
0,1.591737,4.200483,7.832245,"[-1.9364797, -2.75540073, 0.90898967]"
1,5.640802,2.305094,4.606757,"[1.77046974, -0.17350153, -1.99398617]"
2,6.672786,8.483263,2.981881,"[-2.05488716, -0.29381591, -0.89173793]"
3,1.908548,0.147931,1.741693,"[-0.89207197, -0.8143158, -1.36426899]"
4,4.375650,6.837884,1.948188,"[-4.65938123, -0.77685475, -3.07403915]"


In [ ]:
train_df.to_csv('train_df.csv')

In [8]:
# test
# 각 분자의 원자개수, 위치, 힘, 에너지 정보 추출
sequence_test, positions_x, positions_y, positions_z = [], [], [], []

for i in range(len(test)):
    mole = test[i] # 각 분자

    atoms = len(mole) # 원자 개수
    sequence_test.append(atoms)

    position = mole.get_positions() # 원자 위치 정보

    for j in range(len(mole)): # 각 원자에 대해
        atom = mole[j]

        positions_x.append(position[j][0])
        positions_y.append(position[j][1])
        positions_z.append(position[j][2])

test_df = pd.DataFrame({'position_x': positions_x, 'position_y':positions_y, 'position_z':positions_z, 'force':None})
test_df.head()

,position_x,position_y,position_z,force
0,9.671275,8.734431,6.151755,None
1,1.676806,2.238918,5.270450,None
2,10.358608,4.824889,9.174357,None
3,4.370620,5.391541,9.812298,None
4,2.453404,10.449967,9.906622,None


In [ ]:
test_df.to_csv('test_df.csv')

[Force] Hyperparameter Setting

In [9]:
# 하이퍼파라미터
input_size = 3  # feature 개수
hidden_size = 256
output_size = 3 # target 개수
num_epochs = 3
batch_size = 256
learning_rate = 0.001

[Force] Dataset

In [10]:
# 데이터셋 구성
class ForceDataset(Dataset):
    def __init__(self, df, mode='test'):
        self.df = df
        self.mode = mode

    def __len__(self):  # 데이터셋의 총 샘플 수를 반환
        return len(self.df)

    def __getitem__(self, idx): # 데이터셋에서 특정 인덱스에 해당하는 데이터를 반환

        pos_x = self.df.loc[idx, 'position_x']
        pos_y = self.df.loc[idx, 'position_y']
        pos_z = self.df.loc[idx, 'position_z']

        # [pos_x, pos_y, pos_z] 형태의 텐서로 변환
        inputs = torch.tensor([pos_x, pos_y, pos_z], dtype=torch.float32)

        # mode가 'test'가 아닌 경우에는 해당 인덱스의 힘(force) 정보를 label로 할당하고,
        # 해당 샘플의 입력과 레이블을 튜플로 반환
        if not self.mode == 'test':
            label = torch.tensor(self.df.loc[idx, 'force'], dtype=torch.float32)
            return inputs, label
        # mode가 'test'인 경우에는 입력 데이터만 반환
        else:
            return inputs

In [11]:
train_dataset = ForceDataset(train_df, 'train')
test_dataset = ForceDataset(test_df, 'test')

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# shuffle=True : 데이터를 에포크마다 셔플하여 무작위로 배치를 생성하라는 의미
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# shuffle=False : 테스트 데이터의 경우 셔플하지 않고 순서대로 배치를 생성하라는 의미

훈련 데이터셋은 일반화를 위해 무작위로 섞어 모델이 데이터의 순서에 의존하지 않도록 학습시키며, 테스트 데이터셋은 모델의 일반화 능력을 정확히 평가하기 위해 순서대로 배치합니다.

[Force] Model

In [12]:
# Force 모델의 아키텍처인 ForceModel 클래스를 정의
class ForceModel(nn.Module): # nn.Module 클래스를 상속하여 정의
    def __init__(self, input_size, hidden_size):
        super(ForceModel, self).__init__()

        # 여러 레이어를 차례로 적용하는 nn.Sequential 객체
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size), # 선형 변환을 수행하는 레이어로,
                                                # 입력 특성을 은닉층의 차원으로 변환

            nn.BatchNorm1d(hidden_size),        # 배치 정규화 레이어로, 학습 중에 입력 데이터를 정규화

            nn.ReLU(),                          # ReLU(활성화 함수)를 적용하여 비선형성을 도입

            nn.Dropout(0.5),                    # 드롭아웃을 적용하여 모델을 일반화하고 과적합을 방지합니다.
                                                # 50%의 뉴런을 비활성화합니다.

            nn.Linear(hidden_size, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.5),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(64, 3)
        )
    # 입력 데이터를 전달하여 모델의 순전파(forward pass) 연산을 정의
    def forward(self, x): # x : 입력 데이터
        y = self.layers(x)  # y : self.layers(x)로 정의하여
                            # 입력 데이터를 self.layers 시퀀셜 레이어에 전달하여 결과를 계산

        return y

이렇게 정의된 ForceModel 클래스는 입력 데이터를 받아 아키텍처 내에서 정의된 레이어를 통해 변환하고 예측 값을 출력하는 모델을 생성합니다.

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"current device is {device}")

# 모델, 손실 함수, 옵티마이저를 초기화
model = ForceModel(input_size, hidden_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

current device is cpu


[Force] Train

In [14]:
# 모델 학습
print("Training Start!")

model.train() # model.train()을 호출하여 모델을 훈련 모드로 설정
for epoch in range(num_epochs): # num_epochs 변수에 지정된 에포크 수만큼 반복
    print(f"{epoch+1}/{num_epochs} epoch..")  # 현재 몇 번째 에포크인지 출력

    #미니 배치(inputs와 labels) 반복:
    for inputs, labels in tqdm(train_loader): # train_loader로부터
                                              # 미니 배치를 순회하며 학습 진행
        # 옵티마이저 초기화
        optimizer.zero_grad()                 # 옵티마이저의 기울기를 초기화


        # 입력과 레이블 데이터 GPU로 전송
        inputs = inputs.to(device)            # inputs와 labels를 GPU(CPU)로 전송
        labels = labels.to(device)


        # 모델 순전파 및 손실 계산
        outputs = model(inputs)               # 모델의 순전파를 수행하고 예측 값 획득
        loss = criterion(outputs, labels)     # 예측값과 실제 레이블 간의 손실을 계산

        # 역전파 및 옵티마이저 업데이트
        loss.backward()                       # 손실에 대한 역전파를 수행. 파라미터에 대한 기울기가 계산
        optimizer.step()                      # 옵티마이저가 계산된 기울기를 사용하여 모델의 파라미터를 업데이트

print("Training Complete!")

Training Start!
1/3 epoch..


  0%|          | 0/5020 [00:00<?, ?it/s]

2/3 epoch..


  0%|          | 0/5020 [00:00<?, ?it/s]

3/3 epoch..


  0%|          | 0/5020 [00:00<?, ?it/s]

Training Complete!


데이터로더에서 미니 배치를 가져와 모델의 학습을 수행하고, 각 에포크마다 손실을 계산하여 모델의 파라미터를 업데이트합니다. 이러한 과정을 반복하여 모델이 훈련 데이터에 적합해지는 과정을 수행합니다.

[Force] Inference

In [16]:
# 테스트 데이터에 대한 추론을 수행
print("Inference Start!")

# 모델을 평가 모드로 설정
model.eval()

# 빈 리스트 preds 초기화
preds = []

# torch.no_grad() 컨텍스트 내에서 추론
with torch.no_grad(): # 그래디언트 계산 비활성화: 메모리 절약, 속도 높임
    # 미니 배치 순회 및 추론 수행
    for inputs in tqdm(test_loader):
        inputs = inputs.to(device)
        outputs = model(inputs)

        # 예측 값 후처리 및 저장
        pred = outputs.detach().cpu().numpy() # outputs를 NumPy 배열로 변환한 후 pred에 저장
        preds.extend(pred)  # pred : GPU에서 디타치(detach)되어 CPU로 이동한 예측 값
                            # preds.extend(pred) : 모든 미니 배치의 예측 값을 preds 리스트에 추가

print("Inference Complete!")
len(preds)

Inference Start!


  0%|          | 0/1154 [00:00<?, ?it/s]

Inference Complete!


295234

[Force] Submission

In [17]:
test_df['force'] = preds # 예측 결과 저장

In [18]:
# 한 분자가 몇 개의 원자로 이루어져 있는지에 따라 범위를 생성
# 각각 훈련 데이터와 테스트 데이터에 대한 원자 범위를 정의하는 리스트
bundles_train, bundles_test = [], []

flag = 0  # 현재 분자의 원자 범위의 시작 인덱스를 나타내는 변수
for size in sequence_train: # sequence_train : 각 훈련 데이터 분자의 원자 개수를 담은 리스트
    bundles_train.append((flag, flag+size)) #  각 분자의 훈련 데이터 내 시작과 끝 인덱스를 튜플로 추가
    flag += size

flag = 0
for size in sequence_test:  # sequence_test : 각 테스트 데이터 분자의 원자 개수를 담은 리스트
    bundles_test.append((flag, flag+size))
    flag += size

결과적으로, 각 분자의 원자들이 데이터셋 내에서 어떤 범위를 가지는지를 bundles_train 및 bundles_test 리스트에 정의합니다. 이 정보는 나중에 각 분자의 예측 값을 추출하여 정리하는 단계에서 활용될 수 있습니다.






In [21]:
# 각 분자의 힘(Force) 예측 값을 저장할 리스트
preds_force = []

# 각 분자의 원자 범위에 해당하는 힘 예측 값을 추출하여 정리
for start, end in bundles_test: # start, end: 원자 범위의 시작과 끝 인덱스
    preds_force.append(np.vstack(preds[start:end])) # 해당 분자의 힘 예측 값을 수직으로 쌓아
                                                    # 2차원 array로 저장

sample['force'] = preds_force
sample.to_csv('sample.csv')

[Energy] Preprocessing

In [22]:
# 'force' 컬럼의 값을 분해하여 각각의 행으로 만듦
force_df = train_df['force'].apply(pd.Series)
force_df.columns = [f'force_{i}' for i in range(3)]

# 분해한 'force' 컬럼을 추가
train_df = train_df.drop('force', axis=1).join(force_df)

# 'force' 컬럼의 값을 분해하여 각각의 행으로 만듦
force_df = test_df['force'].apply(pd.Series)
force_df.columns = [f'force_{i}' for i in range(3)]

# 분해한 'force' 컬럼을 추가
test_df = test_df.drop('force', axis=1).join(force_df)
test_df.head()


,position_x,position_y,position_z,force_0,force_1,force_2
0,9.671275,8.734431,6.151755,0.007252,0.002667,0.000340
1,1.676806,2.238918,5.270450,0.004389,-0.000588,0.000153
2,10.358608,4.824889,9.174357,0.011033,0.004675,0.002627
3,4.370620,5.391541,9.812298,0.007725,0.002921,0.000784
4,2.453404,10.449967,9.906622,0.007252,0.002667,0.000340


In [23]:
# 데이터프레임에서 값 추출
sequences_train = [train_df.iloc[start:end].values for start, end in bundles_train]
sequences_test = [test_df.iloc[start:end].values for start, end in bundles_test]

[Energy] Hyperparameter Setting

In [24]:
# Energy 모델의 하이퍼파라미터를 설정
input_size = 6  # feature 개수
hidden_size = 256
output_size = 1 # target 개수
num_epochs = 1
batch_size = 64
learning_rate = 0.001

[Energy] Dataset


In [25]:
# 패딩을 사용하여 모든 시퀀스의 길이를 동일하게 만듦
# (Train 데이터를 패딩하여 동일한 길이로 만든 후, 데이터셋을 구성)

# 모든 훈련 데이터 분자의 시퀀스 중 최대 길이
max_len = max(seq.shape[0] for seq in sequences_train)

# 각 분자의 시퀀스 데이터에 대해 패딩을 수행한 결과를 저장.
# 패딩을 통해 모든 시퀀스가 동일한 길이가 됨.
# np.vstack([seq, np.zeros((max_len - seq.shape[0], 6))]) :
# 시퀀스 데이터에 0으로 채워진 패딩을 추가하여 최대 길이로 만드는 과정
padded_sequences = [np.vstack([seq, np.zeros((max_len - seq.shape[0], 6))]) for seq in sequences_train]


# 패딩된 시퀀스를 2차원 배열로 변환
padded_array_train = np.stack(padded_sequences) # 패딩된 시퀀스 데이터를 2차원 배열로 변환
X_tensor_train = torch.tensor(padded_array_train, dtype=torch.float32)  # 패딩된 시퀀스 데이터를 PyTorch 텐서로 변환
y_tensor_train = torch.tensor(energies, dtype=torch.float32).view(-1, 1)  # 에너지 데이터를 PyTorch 텐서로 변환

# TensorDataset : 입력 데이터와 대응하는 레이블을 하나의 데이터셋으로 묶어주는 클래스
train_dataset = TensorDataset(X_tensor_train, y_tensor_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 패딩을 사용하여 모든 시퀀스의 길이를 동일하게 만듦
max_len = max(seq.shape[0] for seq in sequences_test)
padded_sequences = [np.vstack([seq, np.zeros((max_len - seq.shape[0], 6))]) for seq in sequences_test]

# 패딩된 시퀀스를 2차원 배열로 변환
padded_array_test = np.stack(padded_sequences)
X_tensor_test = torch.tensor(padded_array_test, dtype=torch.float32)
test_dataset = TensorDataset(X_tensor_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

[Energy] Model

In [26]:
# BiLSTM 모델 정의
class EnergyModel(nn.Module):
    # 매개변수:
          # input_size: 입력 특성의 크기
          # hidden_size: LSTM의 은닉 상태 크기
          # num_layers: LSTM의 층 수 (기본값: 1)
          # dropout_rate: Dropout 비율 (기본값: 0.5)
    def __init__(self, input_size, hidden_size, num_layers=1, dropout_rate=0.5):
        super(EnergyModel, self).__init__()

        # 모델 내에 두 개의 LSTM 층을 가진 Bidirectional LSTM과 Linear 레이어를 정의
        # Bidirectional LSTM with Dropout
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True,
                            dropout=dropout_rate,
                            bidirectional=True)

        # Bidirectional LSTM의 출력을 받아 에너지 예측을 위한 Linear 레이어를 정의
        # Bidirectional LSTM이므로 hidden_size 조정
        self.linear = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),  # 은닉 상태 크기 두 배 : 양방향 LSTM 출력의 크기에 맞춥
            nn.ReLU(),                                # 비선형성을 추가
            nn.BatchNorm1d(hidden_size),              # 네트워크의 안정성을 높이기 위한 정규화 방법
            nn.Dropout(dropout_rate),                 # 과적합 방지를 위해 뉴런을 무작위로 비활성화
            nn.Linear(hidden_size, 1)                 # 은닉 상태 크기를 최종 에너지 예측값으로 변환
        )


    def forward(self, x): # 모델의 순전파 연산을 정의하는 메서드
        lstm_out, _ = self.lstm(x)  # x : 입력 데이터로, Bidirectional LSTM에 전달
                                    # lstm_out : Bidirectional LSTM의 출력

        # 최종 에너지 예측값은 LSTM 출력의 마지막 타임 스텝에서의 은닉 상태를 Linear 레이어에 적용하여 얻습니다.
        energy = self.linear(lstm_out[:, -1, :])
        return energy

In [27]:
# 모델, 손실 함수, 옵티마이저 초기화
model = EnergyModel(input_size, hidden_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

c:\Users\YEBIN\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


[Energy] Train

In [28]:
print("Training Start!!")

# 학습
model.train()
for epoch in range(num_epochs):
    print(f"{epoch+1}/{num_epochs} epoch..")
    for inputs, labels in tqdm(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

print("Training Complete!")

Training Start!!
1/1 epoch..


  0%|          | 0/352 [00:00<?, ?it/s]

[Energy] Inference

In [ ]:
# 추론 수행
print("Inference Start!")

model.eval()

preds = []
with torch.no_grad():
    for inputs in tqdm(test_loader):
        inputs = inputs[0].to(device)

        outputs = model(inputs)
        pred = outputs.detach().cpu().numpy()

        preds.extend(pred)

print("Inference Complete!")
len(preds)


Inference Start!


  0%|          | 0/65 [00:00<?, ?it/s]

Inference Complete!


4101

[Energy] Submission

In [ ]:
preds = [pred.item() for pred in preds]
sample['energy'] = preds
sample

,ID,energy,force
0,TEST_0000,-61.111519,"[[0.012714688, 0.0064194333, -0.0049114088], [..."
1,TEST_0001,-61.111549,"[[0.012255937, 0.0065041343, -0.0042652227], [..."
2,TEST_0002,-61.111523,"[[0.012147827, 0.006524095, -0.004112943], [0...."
3,TEST_0003,-61.111523,"[[0.012587041, 0.0064430013, -0.0047316076], [..."
4,TEST_0004,-61.111492,"[[0.013105195, 0.006347332, -0.0054614674], [0..."
...,...,...,...
4096,TEST_4096,-61.111599,"[[0.009327458, 0.008304927, -0.00032931505], [..."
4097,TEST_4097,-61.111591,"[[0.012546437, 0.0060378434, -0.0033117067], [..."
4098,TEST_4098,-61.111572,"[[0.012195604, 0.006121706, -0.002880563], [0...."
4099,TEST_4099,-61.111561,"[[0.012258429, 0.0061073904, -0.0029600875], [..."


In [53]:
sample.to_csv('baseline_submission1.csv', index=False)

In [54]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

# 데이터 파일 경로 설정
data_file_path = 'baseline_submission1.csv'

# 데이터 로드
data = pd.read_csv(data_file_path)

# 람다 값 설정
lambda_force = 1 / 25

# RMSE 계산 함수
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# 각 데이터의 energy 및 force 값을 추출
energies = data['energy'].values

In [55]:
# 각 데이터의 force 값을 파싱하여 Numpy 배열로 변환
forces = []
for force_str in data['force']:
    force_values = np.array([list(map(float, row.split())) for row in force_str[2:-2].split(']\n [')])
    forces.append(force_values)
forces = np.array(forces)

<ipython-input-55-7f35983fb888>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  forces = np.array(forces)


In [56]:
data

,ID,energy,force
0,TEST_0000,-61.111519,[[ 0.01271469 0.00641943 -0.00491141]\n [ 0.0...
1,TEST_0001,-61.111549,[[ 1.22559369e-02 6.50413428e-03 -4.26522270e...
2,TEST_0002,-61.111523,[[ 1.21478271e-02 6.52409485e-03 -4.11294308e...
3,TEST_0003,-61.111523,[[ 1.25870407e-02 6.44300133e-03 -4.73160762e...
4,TEST_0004,-61.111492,[[ 1.31051950e-02 6.34733215e-03 -5.46146743e...
...,...,...,...
4096,TEST_4096,-61.111599,[[ 9.32745822e-03 8.30492657e-03 -3.29315051e...
4097,TEST_4097,-61.111591,[[ 0.01254644 0.00603784 -0.00331171]\n [ 0.0...
4098,TEST_4098,-61.111572,[[ 0.0121956 0.00612171 -0.00288056]\n [ 0.0...
4099,TEST_4099,-61.111561,[[ 1.22584291e-02 6.10739039e-03 -2.96008750e...


In [57]:
forces[0]

array([[ 0.01271469,  0.00641943, -0.00491141],
       [ 0.00969585,  0.00728805, -0.00070585],
       [ 0.01242057,  0.00647374, -0.00449712],
       [ 0.01271234,  0.0058617 , -0.00431163],
       [ 0.0067618 ,  0.00901703,  0.00076152],
       [ 0.01150946,  0.00752459, -0.00407458],
       [ 0.01231925,  0.00766691, -0.0062252 ],
       [ 0.01172717,  0.00660176, -0.00352041],
       [ 0.01271934,  0.00694375, -0.0057545 ],
       [ 0.0092185 ,  0.00799198, -0.00012586],
       [ 0.0119774 ,  0.00648075, -0.00379293],
       [ 0.01183201,  0.00650635, -0.00358682],
       [ 0.01453433,  0.00590996, -0.00690155],
       [ 0.01096327,  0.0067428 , -0.0024444 ],
       [ 0.010974  ,  0.00786174, -0.00262771],
       [ 0.01382037,  0.00502784, -0.00519993],
       [ 0.01223404,  0.00634094, -0.00441983],
       [ 0.01260352,  0.00623718, -0.00408519],
       [ 0.01114107,  0.00670998, -0.00269484],
       [ 0.01115418,  0.00670755, -0.00271332],
       [ 0.01088457,  0.00675733, -0.002

In [39]:
# RMSE of per-atom energy 계산
rmse_per_atom_energy = rmse(energies, np.zeros_like(energies))

In [45]:
len(energies)

4101

In [40]:
print(rmse_per_atom_energy)

164.8453750680391


ValueError: ignored

In [41]:
# Axis-wise RMSE of force 계산
axis_wise_rmse_force = np.sqrt(np.sum(np.square(forces), axis=(1, 0)) / forces.shape[0])

AxisError: ignored

In [ ]:

# Axis-wise RMSE of force 계산
axis_wise_rmse_force = np.sqrt(np.sum(np.square(forces), axis=(2, 1)) / forces.shape[0])

# EF metric 계산
ef_metric = rmse_per_atom_energy + lambda_force * np.mean(axis_wise_rmse_force)

# Score 계산
score = ef_metric * 1000

print("EF Metric:", ef_metric)
print("Score:", score)

AxisError: ignored